In [117]:
#IMPORT LIBRARIES
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [118]:
df0=pd.read_csv("datasets/airlinereviews128k.csv", parse_dates=['DateFlown'])

In [119]:
pd.set_option('display.max_columns', None)

In [120]:
df0.sample()

,Aircraft,AirlineName,CabinType,DateFlown,DatePub,EntertainmentRating,FoodRating,GroundServiceRating,OriginCountry,OverallScore,Recommended,Review,Route,SeatComfortRating,ServiceRating,Slug,Title,TravelType,TripVerified,ValueRating,WifiRating,unique_id
57733,NaN,Frontier Airlines,Economy Class,2018-09-01,5th September 2018,0,1,1,UnitedStates,1.0,no,Denver to Providence. Pretty much the worst. 3...,Denver to Providence,1,1,frontier-airlines,absolute worst airline,Couple Leisure,Trip Verified,3,0,f79ca3bc-db0a-4deb-a05a-d9655520660e


In [121]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129455 entries, 0 to 129454
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Aircraft             36423 non-null   object        
 1   AirlineName          129455 non-null  object        
 2   CabinType            126437 non-null  object        
 3   DateFlown            90993 non-null   datetime64[ns]
 4   DatePub              129455 non-null  object        
 5   EntertainmentRating  129455 non-null  int64         
 6   FoodRating           129455 non-null  int64         
 7   GroundServiceRating  129455 non-null  int64         
 8   OriginCountry        127777 non-null  object        
 9   OverallScore         125124 non-null  float64       
 10  Recommended          129455 non-null  object        
 11  Review               128631 non-null  object        
 12  Route                90825 non-null   object        
 13  SeatComfortRat

## EDA

In [122]:
#pass names to uppercase (to differentiate better)
df0.columns = [x.upper() for x in df0.columns]

In [123]:
#keep only dataset with Review values and Title of Reviews
df0=df0.dropna(subset="REVIEW") #too complicated to impute
df0=df0.dropna(subset="TITLE") #too complicated to impute
df0.reset_index(drop=True, inplace=True)

In [124]:
df0.ROUTE.unique() #probably transform it later for origin and destiny

array(['Moroni to Moheli', 'Moroni to Anjouan', 'Anjouan to Dzaoudzi',
       ..., 'Budapest to Göteborg', 'London to Burgas',
       'Paris Beauvais to Riga'], dtype=object)

In [125]:
#keep only dataset with Aircraft values
#df0=df0.dropna(subset="AIRCRAFT")

In [126]:
#clean the tripverified variable
df0.TRIPVERIFIED.value_counts()

TRIPVERIFIED
Trip Verified                                45311
Not Verified                                 13755
NotVerified                                      9
Trip Verified,Trip Verified                      1
Chicago to Colorado Springs. Not Verified        1
Name: count, dtype: int64

In [127]:
df0["TRIPVERIFIED"].replace("Trip Verified,Trip Verified","Trip Verified ", inplace=True)
df0["TRIPVERIFIED"].replace("NotVerified","Not Verified ", inplace=True)
df0["TRIPVERIFIED"].replace("Not Verified,Not Verified","Not Verified ", inplace=True)
df0["TRIPVERIFIED"].replace("Chicago to Colorado Springs. Not Verified","Not Verified ", inplace=True)

df0["TRIPVERIFIED"]=df0.TRIPVERIFIED.str.strip()

In [128]:
#check time horizon to see if enough for time series
print(df0["DATEPUB"].min())
print(df0["DATEPUB"].max())

#ok

10th April 2005
9th September 2022


In [129]:
df0.drop_duplicates()
df0.shape #no duplicates

(128627, 22)

In [130]:
df0.describe()

,DATEFLOWN,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,OVERALLSCORE,SEATCOMFORTRATING,SERVICERATING,VALUERATING,WIFIRATING
count,90166,128627.000000,128627.000000,128627.000000,124297.000000,128627.000000,128627.000000,128627.000000,128627.000000
mean,2018-12-26 22:17:51.144333568,1.559260,1.991557,1.648464,4.563103,2.471518,2.690602,2.639780,0.453132
min,2012-04-01 00:00:00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2017-01-01 00:00:00,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,2018-11-01 00:00:00,1.000000,1.000000,1.000000,3.000000,2.000000,3.000000,2.000000,0.000000
75%,2020-11-01 00:00:00,3.000000,4.000000,3.000000,8.000000,4.000000,4.000000,4.000000,0.000000
max,2023-05-01 00:00:00,5.000000,5.000000,5.000000,10.000000,5.000000,5.000000,5.000000,5.000000
std,NaN,1.751718,1.774979,1.740740,3.499330,1.620989,1.775078,1.629001,1.077762


In [131]:
from functions import nulls

In [132]:
nulls(df0) #most under 3% so ok to impute. Check also if some variables have 0s where missing. So far: assumption that when no service, user set 0

,column_name,nulls_percentage
0,AIRCRAFT,0.718939
1,AIRLINENAME,0.000000
2,CABINTYPE,0.023455
3,DATEFLOWN,0.299012
4,DATEPUB,0.000000
5,ENTERTAINMENTRATING,0.000000
6,FOODRATING,0.000000
7,GROUNDSERVICERATING,0.000000
8,ORIGINCOUNTRY,0.013045
9,OVERALLSCORE,0.033663


Need for imputation:
- AIRCRAFT:won't be used so far, keep it like this for now
- TRAVELTYPE: drop values as I want to keep this var but too many missing to impute efficiently
- DATEFLOWN: ok, impute
- ROUTE: ok, impute
- CABINTYPE: ok, impute
- OVERALLSCORE: ok, impute

In [133]:
#too many missing on traveltype but I want that variable so let's drop null values from it (still enough observ.)

df0=df0.dropna(subset="TRAVELTYPE")

In [134]:
df0.DATEFLOWN.isna().sum() #ok

158

In [135]:
pd.Timestamp.min
df0['DATEFLOWN']=pd.to_datetime(df0['DATEFLOWN'].fillna(pd.Timestamp.min))

In [136]:
#impute using the most common route (mode)
df0['ROUTE'] = df0['ROUTE'].fillna(df0['ROUTE'].mode()[0])

In [137]:
df0.ROUTE.isna().sum() #ok

0

In [138]:
df0['CABINTYPE'] = df0['CABINTYPE'].fillna(df0['CABINTYPE'].mode()[0])

In [139]:
df0.CABINTYPE.isna().sum() #ok

0

In [140]:
nulls(df0)

,column_name,nulls_percentage
0,AIRCRAFT,0.599763
1,AIRLINENAME,0.000000
2,CABINTYPE,0.000000
3,DATEFLOWN,0.000000
4,DATEPUB,0.000000
5,ENTERTAINMENTRATING,0.000000
6,FOODRATING,0.000000
7,GROUNDSERVICERATING,0.000000
8,ORIGINCOUNTRY,0.000000
9,OVERALLSCORE,0.000011


In [141]:
df0.OVERALLSCORE.value_counts()

OVERALLSCORE
1.0     39008
2.0      9363
10.0     9012
9.0      7413
8.0      6775
3.0      5541
7.0      4459
4.0      3099
5.0      2964
6.0      2684
Name: count, dtype: int64

In [142]:
df0['OVERALLSCORE']=df0.OVERALLSCORE.fillna(round(np.mean(df0.OVERALLSCORE))) 

In [143]:
df0.OVERALLSCORE.isna().sum() #ok

0

In [144]:
#TripVerified: assume that missing is 'not verified'

In [145]:
df0['TRIPVERIFIED']=df0.TRIPVERIFIED.fillna("Not Verified") 

In [146]:
df0.TRIPVERIFIED.isna().sum() #ok

0

In [147]:
#for now, in the rating vars asumme that 0 is 0 and non missing (if no service, I put a 0)

In [148]:
df0["OVERALLSCORE"] = df0["OVERALLSCORE"].astype(int)

In [149]:
df0.reset_index(drop=True, inplace=True)

In [150]:
df=df0.drop(columns="AIRCRAFT")

In [151]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90319 entries, 0 to 90318
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   AIRCRAFT             36149 non-null  object        
 1   AIRLINENAME          90319 non-null  object        
 2   CABINTYPE            90319 non-null  object        
 3   DATEFLOWN            90319 non-null  datetime64[ns]
 4   DATEPUB              90319 non-null  object        
 5   ENTERTAINMENTRATING  90319 non-null  int64         
 6   FOODRATING           90319 non-null  int64         
 7   GROUNDSERVICERATING  90319 non-null  int64         
 8   ORIGINCOUNTRY        90319 non-null  object        
 9   OVERALLSCORE         90319 non-null  int32         
 10  RECOMMENDED          90319 non-null  object        
 11  REVIEW               90319 non-null  object        
 12  ROUTE                90319 non-null  object        
 13  SEATCOMFORTRATING    90319 non-

In [152]:
df0.shape

(90319, 22)

In [153]:
df.shape 

(90319, 21)

In [154]:
##CHECK VALUES OF VARIABLES

In [155]:
df.CABINTYPE.value_counts() #ok

CABINTYPE
Economy Class      74836
Business Class     10507
Premium Economy     3360
First Class         1616
Name: count, dtype: int64

In [156]:
df.ENTERTAINMENTRATING.value_counts()

ENTERTAINMENTRATING
0    39937
1    19423
4     8641
3     8480
5     7984
2     5854
Name: count, dtype: int64

In [157]:
df.FOODRATING.value_counts()

FOODRATING
0    26019
1    23418
5    10827
3    10675
4    10594
2     8786
Name: count, dtype: int64

In [158]:
df.GROUNDSERVICERATING.value_counts()

GROUNDSERVICERATING
1    42006
5    15405
4    12225
3     9399
2     7759
0     3525
Name: count, dtype: int64

In [159]:
df.OVERALLSCORE.value_counts()

OVERALLSCORE
1     39008
2      9363
10     9012
9      7413
8      6775
3      5541
7      4459
4      3100
5      2964
6      2684
Name: count, dtype: int64

In [160]:
df.SEATCOMFORTRATING.value_counts()

SEATCOMFORTRATING
1    28506
3    15943
4    14855
2    12681
5    12141
0     6193
Name: count, dtype: int64

In [161]:
df.SERVICERATING.value_counts()

SERVICERATING
1    26623
5    21438
4    12694
3    12484
2    10587
0     6493
Name: count, dtype: int64

In [162]:
df.VALUERATING.value_counts()

VALUERATING
1    41477
5    17176
4    11373
2    10647
3     9632
0       14
Name: count, dtype: int64

In [163]:
df.WIFIRATING.value_counts()

WIFIRATING
0    61095
1    17846
5     3137
3     3048
4     2612
2     2581
Name: count, dtype: int64

In [164]:
df.TRAVELTYPE.value_counts()

TRAVELTYPE
Solo Leisure      32957
Couple Leisure    23611
Family Leisure    19703
Business          14048
Name: count, dtype: int64

In [165]:
df.TRIPVERIFIED.value_counts()

TRIPVERIFIED
Trip Verified    45310
Not Verified     45009
Name: count, dtype: int64

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90319 entries, 0 to 90318
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   AIRLINENAME          90319 non-null  object        
 1   CABINTYPE            90319 non-null  object        
 2   DATEFLOWN            90319 non-null  datetime64[ns]
 3   DATEPUB              90319 non-null  object        
 4   ENTERTAINMENTRATING  90319 non-null  int64         
 5   FOODRATING           90319 non-null  int64         
 6   GROUNDSERVICERATING  90319 non-null  int64         
 7   ORIGINCOUNTRY        90319 non-null  object        
 8   OVERALLSCORE         90319 non-null  int32         
 9   RECOMMENDED          90319 non-null  object        
 10  REVIEW               90319 non-null  object        
 11  ROUTE                90319 non-null  object        
 12  SEATCOMFORTRATING    90319 non-null  int64         
 13  SERVICERATING        90319 non-

## FEATURE ENGINEERING

In [167]:
#Transform the travel type variable in two: type of traveller (solo, family, business) and purpose (leisure, business)

In [168]:
travell_transf=df["TRAVELTYPE"].str.split(" ", expand=True)

In [169]:
travell_transf.rename(columns={travell_transf.columns[0]: 'TRAVELLER'}, inplace=True)
travell_transf.rename(columns={travell_transf.columns[1]: 'PURPOSE'}, inplace=True)

In [170]:
travell_transf.TRAVELLER.value_counts()

TRAVELLER
Solo        32957
Couple      23611
Family      19703
Business    14048
Name: count, dtype: int64

In [171]:
travell_transf["PURPOSE"]=np.where(travell_transf['TRAVELLER']=="Business", "Bussiness",travell_transf.PURPOSE)

In [172]:
travell_transf["TRAVELLER"]=np.where(travell_transf['TRAVELLER']=="Business", "Solo",travell_transf.TRAVELLER) #assume business is 'solo' in traveller_type

In [173]:
df.columns.get_loc('TRAVELTYPE') 

16

In [174]:
df = pd.concat([df.iloc[:, :17], travell_transf, df.iloc[:, 17:]], axis=1)

In [175]:
df.drop(columns='TRAVELTYPE', inplace=True)

In [176]:
df.RECOMMENDED.value_counts() #quite imbalanced. Need to balance it?

RECOMMENDED
no     59309
yes    31010
Name: count, dtype: int64

In [177]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
5509,Air Koryo,Economy Class,2015-07-01,8th September 2015,0,3,3,Thailand,6,yes,"If you compare Air China with Air Koryo, there...",PEK to FNJ,3,4,air-koryo,good old-style music,Solo,Bussiness,Not Verified,3,0,5d97460b-acff-46cd-a099-63ee3fcb12b4


In [178]:
#clean variable #ROUTE

In [179]:
route_origin = df["ROUTE"].str.split(r"\bto\b", expand=True)

In [180]:
route_origin.head()

,0,1,2,3,4
0,Moroni,Moheli,None,None,None
1,Moroni,Anjouan,None,None,None
2,Anjouan,Dzaoudzi,None,None,None
3,Milano,Olbia,None,None,None
4,Cascais,Bragança,None,None,None


In [181]:
route_origin.reset_index(drop=True, inplace=True)

In [182]:
route_origin.info() #will assume that other 'to' are not necessary (writing typos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90319 entries, 0 to 90318
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       90319 non-null  object
 1   1       90113 non-null  object
 2   2       57 non-null     object
 3   3       6 non-null      object
 4   4       1 non-null      object
dtypes: object(5)
memory usage: 3.4+ MB


In [183]:
df.ROUTE.unique() #clearly origin and destiny separated by 'to'

array(['Moroni to Moheli', 'Moroni to Anjouan', 'Anjouan to Dzaoudzi',
       ..., 'Budapest to Göteborg', 'London to Burgas',
       'Paris Beauvais to Riga'], dtype=object)

In [184]:
route_origin.reset_index(drop=True, inplace=True)

In [185]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
53420,Lufthansa,Economy Class,2017-08-01,28th August 2017,5,5,1,UnitedStates,1,no,Athens to Munich. We boarded almost an hour la...,Athens to Munich,2,5,lufthansa,My bag didn't make it,Family,Leisure,Not Verified,2,4,68918957-ea37-45b0-ae57-22e897230c4b


In [186]:
df.reset_index(drop=True, inplace=True) #important to reset index or won't fit the new variables with route

In [187]:
df.columns.get_loc('ROUTE')

11

In [188]:
df = pd.concat([df.iloc[:, :12], route_origin, df.iloc[:, 12:]], axis=1)

In [189]:
pd.set_option('display.max_columns', None) #to check all columns when sampling the df

In [190]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,0,1,2,3,4,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
83204,Turkish Airlines,Business Class,2020-02-01,15th February 2020,0,0,1,UnitedStates,1,no,Dar es Salaam to Washington via Istanbul. Init...,Dar es Salaam to Washington via Istanbul,Dar es Salaam,Washington via Istanbul,None,None,None,1,1,turkish-airlines,zero concept of customer service,Solo,Leisure,Trip Verified,1,0,5544ce23-ccc5-489d-a4a6-c4dd7c4a4bca


In [191]:
df.drop(columns=df.columns[15], inplace=True) #3 times, to take the extra 'tos', we are goint to assume there is only a 'to'

In [192]:
df.drop(columns=df.columns[15], inplace=True) 

In [193]:
df.drop(columns=df.columns[14], inplace=True) 

In [194]:
df.rename(columns={df.columns[12]: 'ORIGIN'}, inplace=True)
df.rename(columns={df.columns[13]: 'DESTINY'}, inplace=True)

In [195]:

df.reset_index(drop=True, inplace=True)

In [196]:
route_destiny = df["DESTINY"].str.split(r"\bvia\b", expand=True)

In [197]:
route_destiny.sample() #if scale is 'none' this means no scale was done. 

,0,1,2
81728,London,None,None


In [198]:
route_destiny.rename(columns={route_destiny.columns[0]: 'DESTINY2'}, inplace=True)
route_destiny.rename(columns={route_destiny.columns[1]: 'SCALE'}, inplace=True)
route_destiny.rename(columns={route_destiny.columns[2]: 'scale2'}, inplace=True)

In [199]:
route_destiny.scale2.value_counts() #very few, we assume only if scale or not

scale2
Paris          1
HKG            1
Mexico City    1
Name: count, dtype: int64

In [200]:
route_destiny.drop(columns="scale2", inplace=True)

In [201]:
route_destiny.reset_index(drop=True, inplace=True)

In [202]:
df.columns.get_loc('DESTINY')

13

In [203]:
df = pd.concat([df.iloc[:, :14], route_destiny, df.iloc[:, 14:]], axis=1)

In [204]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,ORIGIN,DESTINY,DESTINY2,SCALE,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
45593,Delta Air Lines,Economy Class,2022-03-01,4th April 2022,0,1,1,UnitedStates,1,no,I booked a flight from Providence to Houston a...,Providence to Houston via New York,Providence,Houston via New York,Houston,New York,1,1,delta-air-lines,the worst airline to fly with,Solo,Bussiness,Trip Verified,1,0,59d455c5-c566-422f-bc0b-de8e34eb452f


In [205]:
df.drop(columns="DESTINY", inplace=True)

In [206]:
df.rename(columns={'DESTINY2':'DESTINY'}, inplace=True)

In [207]:
df.reset_index(drop=True, inplace=True) 

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90319 entries, 0 to 90318
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   AIRLINENAME          90319 non-null  object        
 1   CABINTYPE            90319 non-null  object        
 2   DATEFLOWN            90319 non-null  datetime64[ns]
 3   DATEPUB              90319 non-null  object        
 4   ENTERTAINMENTRATING  90319 non-null  int64         
 5   FOODRATING           90319 non-null  int64         
 6   GROUNDSERVICERATING  90319 non-null  int64         
 7   ORIGINCOUNTRY        90319 non-null  object        
 8   OVERALLSCORE         90319 non-null  int32         
 9   RECOMMENDED          90319 non-null  object        
 10  REVIEW               90319 non-null  object        
 11  ROUTE                90319 non-null  object        
 12  ORIGIN               90319 non-null  object        
 13  DESTINY              90113 non-

In [209]:
df= df.dropna(axis=0, subset=['DESTINY']) # too complicated to clean and separate, so drop (only 37 values)

In [210]:
#delete spaces before and after
df["ORIGIN"]=df.ORIGIN.str.strip()
df["DESTINY"]=df.DESTINY.str.strip()

In [211]:
df.insert(15,'SCALE_YN',np.where(df['SCALE'].isna(), 0,1))

In [212]:
df.drop(columns="SCALE", inplace=True)

In [213]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,ORIGIN,DESTINY,SCALE_YN,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
45528,Delta Air Lines,Economy Class,2022-05-01,2nd May 2022,3,1,4,UnitedStates,2,no,Flight was cancelled because Delta didn't sche...,Atlanta to Memphis,Atlanta,Memphis,0,2,4,delta-air-lines,$75 for 10 hours!,Solo,Leisure,Not Verified,1,2,29b8075e-2432-403a-8758-bdabbfe27a7b


In [214]:
#create a variable that states whether the flight was overseas or not. If entertainment diff. from 0 -->overseas; otherwise, no.
df['LONGDISTANCE']=np.where(df.ENTERTAINMENTRATING==0,0,1)

In [215]:
df['TRIPVERIFIED']=np.where(df['TRIPVERIFIED']=="Trip Verified", 1,0) 

In [216]:
df.RECOMMENDED.value_counts() #quite imbalanced...better check later balancing with SMOTE?

RECOMMENDED
no     59224
yes    30889
Name: count, dtype: int64

In [217]:
df_categ=df.copy()

In [218]:
#pass RECOMMENDED TO DUMMY
df["RECOMMENDED"]=np.where(df["RECOMMENDED"]=="yes",1,0)

In [219]:
#change categoricals string to categorical numbers 1=lowest class and 4=highest class
df['CABINTYPE']=np.where(df['CABINTYPE']=="Economy Class", 1,df.CABINTYPE)
df['CABINTYPE']=np.where(df['CABINTYPE']=="Premium Economy", 2,df.CABINTYPE)
df['CABINTYPE']=np.where(df['CABINTYPE']=="Business Class", 3,df.CABINTYPE) #in order to avoid such an inbalanced class (firstclass few n)
df['CABINTYPE']=np.where(df['CABINTYPE']=="First Class", 4,df.CABINTYPE)

In [220]:
#pass froms string to int
df["CABINTYPE"]=df["CABINTYPE"].astype(int)

In [221]:
df.TRAVELLER.value_counts()

TRAVELLER
Solo      46871
Couple    23569
Family    19673
Name: count, dtype: int64

In [222]:
#change the variable traveller to numeric (based on specific order)
df['TRAVELLER']=np.where(df['TRAVELLER']=="Solo", 1,df.TRAVELLER)
df['TRAVELLER']=np.where(df['TRAVELLER']=="Couple", 2,df.TRAVELLER)
df['TRAVELLER']=np.where(df['TRAVELLER']=="Family", 3,df.TRAVELLER)

In [223]:
#pass froms string to int
df["TRAVELLER"]=df["TRAVELLER"].astype(int)

In [224]:
df.PURPOSE.value_counts()

PURPOSE
Leisure      76103
Bussiness    14010
Name: count, dtype: int64

In [225]:
#change the variable traveller to numeric (based on specific order)
df['PURPOSE']=np.where(df['PURPOSE']=="Leisure", 1,0) 

In [226]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,ORIGIN,DESTINY,SCALE_YN,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID,LONGDISTANCE
8415,Air India,1,2018-04-01,23rd April 2018,0,1,2,Singapore,2,0,Singapore - Mumbai and New Delhi - Singapore. ...,New Delhi to Singapore,New Delhi,Singapore,0,2,2,air-india,far from acceptable standards,1,1,1,3,0,17abf4c9-faaa-48df-9636-11a57d13691f,0


In [227]:
df['YEAR'] = df['DATEFLOWN'].dt.year

In [228]:
df.shape

(90113, 27)

In [229]:
df.YEAR.value_counts() #1677 makes no sense! drop these 105 values

YEAR
2019    15199
2018    14040
2022    12142
2016    11847
2017    11481
2015     9734
2021     6329
2020     5529
2023     3557
2014      147
1677      105
2013        2
2012        1
Name: count, dtype: int64

In [230]:
df = df[(df.YEAR > 1677)] #avoid negative values --> 1677

In [231]:
##CLEAN DATABASE!
df.shape

(90008, 27)

In [232]:
df_eda_clean=df.to_csv("datasets/cleandf.csv")

In [117]:
df_eda_clean=df.copy()